In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
import helper
import solution
from IPython.display import HTML

import sys
sys.path.append('../')
import commons

In [3]:
def are_equal(a, b):
    if isinstance(a, (int, float)):
        return (abs(a-b) < 1e-8)
    elif isinstance(a, np.ndarray):
        return (np.allclose(a, b))
    elif isinstance(a, dict):
        common = set(a) & set(b)
        return all([are_equal(a[k], b[k]) for k in common])
    else:
        if len(a) != len(b):
            return False
        return all([are_equal(a[i], b[i]) for i in range(len(a))])
    

In [4]:
data = loadmat('./data/PA8SampleCases.mat', simplify_cells=True)
inp = data['exampleINPUT']
out = data['exampleOUTPUT']

for k in ['t3a2', 't4a2', 't5a4']: # these are G structures but in with 1 based indexes. Fixing them.
    inp[k][:, 1] -= 1
    
out['t7o2'][:, 1, :] -=1
out['t7o2'][0, 1, :] = 0

# Now all indexes are zero based.

## Fit Gaussian Parameters

In [5]:
mu, sigma = solution.fit_gaussian_parameters(inp['t1a1'])
print(are_equal(mu, out['t1o1']))
print(are_equal(sigma, out['t1o2']))

True
True


## Fit Linear Gaussian Parameters

In [6]:
beta, sigma = solution.fit_linear_gaussian_parameters(inp['t2a1'], inp['t2a2'])
print(are_equal(beta, out['t2o1']))
print(are_equal(sigma, out['t2o2']))

True
True


## Model Learning

In [7]:
data2 = loadmat('./data/PA8Data.mat', simplify_cells=True)

train_x = data2['trainData']['data']
train_y = data2['trainData']['labels']
test_x = data2['testData']['data']
test_y = data2['testData']['labels']
train_is_human = (train_y[:, 0] == 1)
train_is_alien = (train_y[:, 1] == 1)
train_human_poses = train_x[train_is_human]
train_alien_poses = train_x[train_is_alien]
human_imgs = helper.visualize_dataset(train_human_poses)
alien_imgs = helper.visualize_dataset(train_alien_poses)

G1 = data2['G1']
G2 = data2['G2']
G1[:, 1] -= 1
G2[:, 1] -= 1

In [8]:
HTML(helper.create_html5_animation(human_imgs, alien_imgs, labels=['Human', 'Alien']))

## Compute Log Likelyhood

In [9]:
are_equal(out['t3'], solution.compute_log_likelihood(inp['t3a1'], inp['t3a2'], inp['t3a3']))

True

## Learn CPDs Given Graph

In [10]:
P, loglikelyhood = solution.learn_cpd_given_graph(inp['t4a1'], inp['t4a2'], inp['t4a3'])

In [11]:
print(are_equal(P, out['t4o1']))
print(are_equal(loglikelyhood, out['t4o2']))

True
True


## Classify Datasets

In [12]:
acc = solution.classify_dataset(inp['t5a1'], inp['t5a2'], inp['t5a3'], inp['t5a4'])

In [13]:
are_equal(acc, out['t5'])

True

### Naive Bayes

In [14]:
P1, l1 = solution.learn_cpd_given_graph(train_x, G1, train_y)
acc1 = solution.classify_dataset(test_x, test_y, P1, G1)
l1, acc1

(-46680.63273953179, 0.79)

In [15]:
sample1 = [helper.visualize_models(P1, G1) for _ in range(100)]
humans = [s[0] for s in sample1]
aliens = [s[1] for s in sample1]

In [16]:
HTML(helper.create_html5_animation(humans, aliens, labels=['Human', 'Alien']))

### Tree Structured

In [17]:
P2, l2 = solution.learn_cpd_given_graph(train_x, G2, train_y)
acc2 = solution.classify_dataset(test_x, test_y, P2, G2)
l2, acc2

(-27716.90852437847, 0.84)

In [31]:
sample2 = [helper.visualize_models(P2, G2) for _ in range(100)]
humans2 = [s[0] for s in sample2]
aliens2 = [s[1] for s in sample2]

In [32]:
HTML(helper.create_html5_animation(humans2, aliens2, labels=['Human', 'Alien']))

# Learning Graph Structures

In [20]:
A, W = solution.learn_graph_structure(inp['t6a1'])

In [21]:
print(are_equal(A, out['t6o1']))
print(are_equal(W, out['t6o2']))

True
True


## Learn Graph and CPD

In [22]:
P3, G3, L3 = solution.learn_graph_and_cpds(inp['t7a1'], inp['t7a2'])

In [23]:
print(are_equal(P3, out['t7o1']))
print(are_equal(G3, out['t7o2']))
print(are_equal(L3, out['t7o3']))

True
True
True


In [25]:
acc3 = solution.classify_dataset(test_x, test_y, P3, G3)
L3, acc3

(-11983.826618636365, 0.94)

In [29]:
sample3 = [helper.visualize_models(P3, G3) for _ in range(100)]
humans3 = [s[0] for s in sample3]
aliens3 = [s[1] for s in sample3]

In [30]:
HTML(helper.create_html5_animation(humans3, aliens3, labels=['Human', 'Alien']))

In [33]:
HTML(helper.create_html5_animation(aliens2, aliens3, labels=['Alien-2', 'Alien-3']))